# AV-PINO Motor Fault Diagnosis - Getting Started

This notebook demonstrates the basic usage of the Adaptive Variational Physics-Informed Neural Operator (AV-PINO) system for motor fault diagnosis.

## Overview

The AV-PINO system combines:
- Physics-informed neural operators
- Multi-physics coupling (electromagnetic, thermal, mechanical)
- Uncertainty quantification
- Real-time inference capabilities

## Prerequisites

Make sure you have installed the required dependencies:
```bash
pip install -r requirements.txt
```

In [ ]:
# Import required libraries
import sys
import os
sys.path.append('../src')

import numpy as np
import matplotlib.pyplot as plt
import torch
from pathlib import Path

# Import AV-PINO components
from config.experiment_config import ExperimentManager
from data.cwru_loader import CWRUDataLoader
from data.signal_processor import SignalProcessor
from physics.feature_extractor import PhysicsFeatureExtractor
from visualization.visualization_manager import VisualizationManager

print("AV-PINO system imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 1. Configuration Setup

First, let's set up the experiment configuration for reproducible results.

In [ ]:
# Initialize experiment manager
config_manager = ExperimentManager()

# Create default configuration
config = config_manager.create_default_config()

# Customize configuration for this demo
config.training["epochs"] = 50  # Reduced for demo
config.training["batch_size"] = 16
config.model["width"] = 32  # Smaller model for demo

# Setup reproducibility
final_config = config_manager.setup_experiment()

print("Configuration setup complete!")
print(f"Random seed: {final_config.reproducibility.seed}")
print(f"Model architecture: {final_config.model['architecture']}")
print(f"Training epochs: {final_config.training['epochs']}")

## 2. Data Loading and Preprocessing

Load the CWRU bearing fault dataset and perform initial preprocessing.

In [ ]:
# Initialize data loader
data_loader = CWRUDataLoader()

# Load sample data (this will download CWRU dataset if not present)
try:
    train_data, test_data = data_loader.load_dataset(
        data_path="../data/cwru",
        download=True,
        test_split=0.2
    )
    print(f"Training samples: {len(train_data)}")
    print(f"Test samples: {len(test_data)}")
    
    # Get a sample for demonstration
    sample = train_data[0]
    print(f"Sample signal shape: {sample.signal.shape}")
    print(f"Sample fault type: {sample.fault_type}")
    
except Exception as e:
    print(f"Data loading failed: {e}")
    print("Creating synthetic data for demonstration...")
    
    # Create synthetic data for demo
    from data.cwru_loader import CWRUDataSample
    
    # Generate synthetic vibration signal
    t = np.linspace(0, 1, 1024)
    signal = np.sin(2 * np.pi * 60 * t) + 0.5 * np.sin(2 * np.pi * 120 * t) + 0.1 * np.random.randn(1024)
    
    sample = CWRUDataSample(
        signal=signal,
        fault_type="inner_race",
        fault_size=0.007
    )
    
    print(f"Synthetic sample created: {sample.signal.shape}")

## 3. Signal Processing and Feature Extraction

Process the raw vibration signal and extract physics-based features.

In [ ]:
# Initialize signal processor
signal_processor = SignalProcessor(
    sampling_rate=12000,  # CWRU dataset sampling rate
    window_size=1024
)

# Process the signal
processed_signal = signal_processor.process_signal(sample.signal)

print("Signal processing complete!")
print(f"Time domain features: {processed_signal.time_features.shape}")
print(f"Frequency domain features: {processed_signal.freq_features.shape}")
print(f"Time-frequency features: {processed_signal.time_freq_features.shape}")

# Extract physics-based features
physics_extractor = PhysicsFeatureExtractor()
physics_features = physics_extractor.extract_features(sample.signal)

print(f"\nPhysics features extracted:")
print(f"Electromagnetic features: {physics_features.electromagnetic.shape}")
print(f"Thermal features: {physics_features.thermal.shape}")
print(f"Mechanical features: {physics_features.mechanical.shape}")

## 4. Visualization

Visualize the processed signals and extracted features.

In [ ]:
# Create visualization plots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot raw signal
axes[0, 0].plot(sample.signal[:1000])
axes[0, 0].set_title('Raw Vibration Signal')
axes[0, 0].set_xlabel('Sample')
axes[0, 0].set_ylabel('Amplitude')

# Plot frequency spectrum
freqs = np.fft.fftfreq(len(sample.signal), 1/12000)
fft_signal = np.abs(np.fft.fft(sample.signal))
axes[0, 1].plot(freqs[:len(freqs)//2], fft_signal[:len(fft_signal)//2])
axes[0, 1].set_title('Frequency Spectrum')
axes[0, 1].set_xlabel('Frequency (Hz)')
axes[0, 1].set_ylabel('Magnitude')
axes[0, 1].set_xlim(0, 1000)

# Plot time domain features
axes[1, 0].plot(processed_signal.time_features)
axes[1, 0].set_title('Time Domain Features')
axes[1, 0].set_xlabel('Feature Index')
axes[1, 0].set_ylabel('Value')

# Plot physics features
all_physics_features = np.concatenate([
    physics_features.electromagnetic,
    physics_features.thermal,
    physics_features.mechanical
])
axes[1, 1].bar(range(len(all_physics_features)), all_physics_features)
axes[1, 1].set_title('Physics-Based Features')
axes[1, 1].set_xlabel('Feature Index')
axes[1, 1].set_ylabel('Value')

plt.tight_layout()
plt.show()

print(f"Fault type: {sample.fault_type}")
print(f"Fault size: {sample.fault_size} inches")

## 5. Model Architecture Overview

Let's examine the neural operator architecture components.

In [ ]:
from physics.spectral_operator import SpectralOperator
from physics.constraints import PhysicsConstraintLayer

# Initialize spectral operator
spectral_op = SpectralOperator(
    n_modes=final_config.model["modes"],
    width=final_config.model["width"]
)

# Initialize physics constraints
physics_constraints = PhysicsConstraintLayer(
    constraint_types=["maxwell", "heat", "structural"]
)

print("Neural operator components initialized:")
print(f"Spectral operator modes: {spectral_op.n_modes}")
print(f"Model width: {spectral_op.width}")
print(f"Physics constraints: {physics_constraints.constraint_types}")

# Create sample input for architecture demonstration
sample_input = torch.randn(1, 1, 1024)  # Batch, channels, sequence length

print(f"\nSample input shape: {sample_input.shape}")
print("Model ready for training and inference!")

## 6. Physics Constraints Demonstration

Demonstrate how physics constraints are incorporated into the model.

In [ ]:
# Demonstrate physics constraint computation
from physics.constraints import MaxwellConstraint, HeatEquationConstraint, StructuralDynamicsConstraint

# Initialize individual constraints
maxwell_constraint = MaxwellConstraint()
heat_constraint = HeatEquationConstraint()
structural_constraint = StructuralDynamicsConstraint()

# Create sample physics fields
sample_fields = {
    'electromagnetic': torch.randn(1, 32),
    'thermal': torch.randn(1, 32),
    'mechanical': torch.randn(1, 32)
}

# Compute constraint residuals
maxwell_residual = maxwell_constraint.compute_residual(
    sample_fields['electromagnetic'], 
    sample_fields['electromagnetic']
)

heat_residual = heat_constraint.compute_residual(
    sample_fields['thermal'],
    sample_fields['thermal']
)

structural_residual = structural_constraint.compute_residual(
    sample_fields['mechanical'],
    sample_fields['mechanical']
)

print("Physics constraint residuals:")
print(f"Maxwell residual: {maxwell_residual.item():.6f}")
print(f"Heat equation residual: {heat_residual.item():.6f}")
print(f"Structural dynamics residual: {structural_residual.item():.6f}")

# Visualize constraint violations
residuals = [maxwell_residual.item(), heat_residual.item(), structural_residual.item()]
constraint_names = ['Maxwell', 'Heat Equation', 'Structural Dynamics']

plt.figure(figsize=(10, 6))
plt.bar(constraint_names, residuals)
plt.title('Physics Constraint Residuals')
plt.ylabel('Residual Value')
plt.yscale('log')
plt.show()

print("\nPhysics constraints help ensure the model learns physically consistent representations!")

## 7. Next Steps

This notebook demonstrated the basic components of the AV-PINO system. To continue exploring:

1. **Training**: See `02_model_training.ipynb` for complete training pipeline
2. **Inference**: See `03_real_time_inference.ipynb` for real-time prediction
3. **Validation**: See `04_model_validation.ipynb` for comprehensive evaluation
4. **Visualization**: See `05_advanced_visualization.ipynb` for detailed analysis

## Summary

In this notebook, we:
- Set up reproducible experiment configuration
- Loaded and preprocessed motor vibration data
- Extracted physics-based features
- Demonstrated neural operator components
- Showed physics constraint integration

The AV-PINO system is now ready for training and deployment!

In [ ]:
print("🎉 AV-PINO Getting Started tutorial complete!")
print("Ready to proceed with model training and inference.")